In [1]:
import pandas as pd

In [19]:
df_demo = pd.read_csv("../anonymized_Labels_refeeding.csv", sep='\t')
df_lab = pd.read_csv("../anonymized_Labels_refeeding_lab.csv", sep='\t')
df_vitals = pd.read_csv("../anonymized_Labels_refeeding_metingen.csv", sep='\t' )


In [20]:
df_lab

,AANVRAAG_NUMMER,DT_BEPALING,NORMAALWAARDE,O_AANVR_UITSLAG_ITEM_LANG,O_STATUS_UITSLAG,SEQ_ZPAT_PATIENT,STATUS_AANVRAAG,UITSLAGREGEL,UITSLAG_CONCLUSIE,UITSLAG_TEKST_LAB,UITSLAG_WAARDE,ggzob_id,intid
0,222999.0,09-02-2016 16:03,NaN,EDE-Q Protocol,Gefiatteerd,4.576214e+08,RB,37716929,Eetstoornissen,NaN,NaN,547117,NaN
1,396868.0,26-10-2016 17:15,NaN,EDE-Q Protocol,Gefiatteerd,4.576214e+08,RB,41877252,Eetstoornissen,NaN,NaN,596340,NaN
2,1247756.0,30-11-2018 22:29,NaN,EDE-Q Protocol,Gecorrigeerde uitslag,7.414413e+07,RB,77581741,Eetstoornissen,NaN,NaN,787361,NaN
3,1281516.0,04-01-2019 10:47,NaN,EDE-Q Protocol,Gecorrigeerde uitslag,4.576214e+08,RB,77430989,Eetstoornissen,NaN,NaN,825641,NaN
4,1349356.0,27-02-2019 19:25,NaN,EDE-Q Protocol,Gecorrigeerde uitslag,7.414413e+07,RB,77443979,Eetstoornissen,NaN,NaN,787361,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vanwege de consistentie is ook onderzoek op Ae...,NaN
23387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gebracht,NaN
23388,1622375.0,26-01-2023 08:36,NaN,Eerdere infectie Hepatitits E,Gefiatteerd,7.414413e+07,RB,75529794,Geen aanwijzing voor eerdere of recente infect...,NaN,NaN,787361,NaN
23389,1686395.0,03-03-2024 15:24,0 - 3.3,Non-HDL_cholesterol,Gefiatteerd,4.439351e+08,RB,79656908,NaN,NaN,Geen res.,893618,NaN
